In [9]:
# combining everything we have 
import cv2;
import numpy as np;
from matplotlib import pyplot as plt;
from pywt import dwt2, idwt2;
import pywt;

def find_neighbors(index, dim):
    all_neighbors = []
    x, y = index
    width, height = dim
    
    all_neighbors.append((x, y - 1))
    all_neighbors.append((x, y + 1))
    all_neighbors.append((x - 1, y))
    all_neighbors.append((x + 1, y))
    all_neighbors.append((x - 1, y - 1))
    all_neighbors.append((x + 1, y - 1))
    all_neighbors.append((x - 1, y + 1))
    all_neighbors.append((x + 1, y + 1))

    neighbors = [index]
    #     save the location that the pixel is on for comparison
    for i in all_neighbors:
        if (i[0] >= width) or (i[1] >= height):
            continue
        elif (i[0] >= 0) and (i[1] >= 0):
            neighbors.append(i)
    return neighbors

# generates the prewitt kernels which will give us the approximate second derivative of gaussian 
def kernelGen(dim, isX):
    if(isX):
        kernelx = np.ones((1,dim), dtype = int) * -1
        zeros = np.zeros((1,dim), dtype = int)
        ones = np.ones((1,dim), dtype = int)
        for num in range(0,dim-2):
            kernelx = np.concatenate((kernelx, zeros), axis = -1)
        kernelx = np.concatenate((kernelx, ones), axis = -1)
        kernelx = np.resize(kernelx, (dim,dim))
        return kernelx
    else:
        kernely =  np.ones((dim,1), dtype = int) * -1
        zeros = np.zeros((dim,1), dtype = int)
        ones = np.ones((dim,1), dtype = int)
        for num in range(0,dim-2):
            kernely = np.concatenate((kernely, zeros), axis = -1)
        kernely = np.concatenate((kernely, ones), axis = -1)
        kernely = np.resize(kernely, (dim,dim))
        return kernely
    
def SURF():
    img = cv2.imread('red_panda.jpg')
    # convert image to grayscale 
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    fSize = [9, 15, 21, 27]
    
    ndet = [] # list of numpy arrays of dim 300*300
    for size in fSize:
        kernelx = kernelGen(size, True)
        kernely = kernelGen(size, False)

        Dx = cv2.filter2D(gray, -1, kernelx)
        Dxx = cv2.filter2D(Dx, -1, kernelx)
        Dy = cv2.filter2D(gray, -1, kernely)
        Dyy = cv2.filter2D(Dy, -1, kernely)
        Dxy = cv2.filter2D(Dx, -1, kernely)
        
        # resize in order to evaluate determinant
        new_Dyy = cv2.resize(Dyy, (300,500))
        new_Dxy = cv2.resize(Dxy, (300,500))
        
        # evaluate determinant based equation given from paper
        determinant = np.dot(Dxx,new_Dyy)-(np.square(.9)*np.dot(Dxy, new_Dxy)) 
        ndet.append(determinant)
    
    det = [] # list of determinant list 
    for i in range(0, len(ndet)):
        det.append(ndet[i].tolist())
    
    kp = [] # stores key points
    loc = [] # stores location of key point
    # now for the non-maximum suppression in 3*3*3 neighborhood between scales
    for index in range(0, len(det)):
        if(index == 0 or index == len(det) - 1):
            continue
        instance = det[index] # 300 * 300 square matrix
        width = len(instance[1])
        height = len(instance[0])
        instance_dim = (width, height)
        for row in range(width):
            for col in range(height):
                neighbors_to_check = []
                current = instance[row][col] 
                upper = det[index+1]
                lower = det[index-1]
                neighbors = find_neighbors((row, col), instance_dim)
                # retrieves all valid neighbors to check, upper & lower neighbor check positions will be exactly the same
                for position in neighbors:
                    neighbors_to_check.append(upper[position[0]][position[1]])
                    neighbors_to_check.append(lower[position[0]][position[1]])
                    neighbors_to_check.append(instance[position[0]][position[1]])
                # node in same position in upper and lower level also needs to be compared
                neighbors_to_check.append(upper[row][col])
                neighbors_to_check.append(lower[row][col])
                if not (current == max(neighbors_to_check)):
                    continue
                else:
                    kp.append(current)
                    loc.append((row,col))
        print('Length of keypoint locations found', len(loc))
        print('Length of location set', len(set(loc)))
        print('Location keypoints:', set(loc))
    
SURF()

Length of keypoint locations found 3331
Length of location set 3331
Location keypoints: {(50, 96), (199, 58), (257, 77), (205, 271), (188, 246), (10, 271), (11, 90), (33, 41), (115, 94), (298, 95), (26, 120), (117, 185), (261, 105), (194, 28), (197, 241), (199, 269), (220, 182), (222, 11), (224, 177), (208, 283), (125, 271), (17, 209), (71, 290), (146, 42), (42, 235), (278, 152), (48, 254), (66, 245), (49, 139), (288, 167), (200, 9), (2, 214), (93, 151), (113, 182), (243, 109), (25, 162), (154, 13), (45, 223), (119, 269), (267, 173), (237, 269), (2, 73), (61, 63), (44, 25), (19, 96), (40, 230), (183, 59), (42, 67), (242, 299), (226, 209), (174, 298), (160, 26), (87, 10), (287, 9), (288, 79), (223, 17), (260, 45), (211, 291), (87, 251), (163, 132), (238, 125), (168, 217), (46, 30), (88, 272), (217, 249), (36, 80), (238, 236), (276, 92), (249, 200), (104, 240), (176, 152), (273, 200), (6, 92), (227, 202), (83, 132), (66, 84), (101, 82), (232, 159), (144, 187), (1, 137), (19, 214), (55, 1

Length of keypoint locations found 6686
Length of location set 6686
Location keypoints: {(43, 3), (232, 72), (33, 41), (124, 110), (20, 273), (298, 95), (261, 105), (100, 199), (194, 28), (213, 288), (199, 269), (75, 259), (146, 42), (42, 235), (65, 154), (66, 245), (288, 167), (142, 6), (113, 182), (94, 287), (288, 278), (237, 269), (61, 63), (40, 230), (131, 49), (60, 249), (226, 209), (107, 190), (269, 99), (259, 149), (122, 273), (168, 217), (23, 197), (46, 30), (88, 272), (268, 44), (36, 80), (224, 208), (83, 132), (230, 117), (162, 269), (1, 137), (55, 122), (92, 78), (96, 9), (25, 3), (12, 271), (72, 226), (135, 83), (233, 49), (267, 219), (6, 196), (171, 263), (280, 30), (176, 159), (125, 212), (185, 167), (232, 134), (171, 37), (180, 285), (218, 72), (43, 111), (261, 60), (63, 96), (289, 163), (80, 232), (66, 55), (123, 74), (86, 50), (109, 115), (237, 62), (167, 279), (42, 60), (160, 47), (51, 262), (142, 115), (217, 50), (109, 4), (271, 277), (189, 172), (62, 138), (156, 91)